
:::{.callout-tip} 
We recommend reading the LCEL [conceptual guide](/docs/concepts#langchain-expression-language-lcel) first.
:::

LCEL is designed to streamline the process of building useful apps with LLMs and combining related components. It does this by providing:

1. **A unified interface**: Every LCEL object implements the `Runnable` interface, which defines a common set of invocation methods (`invoke`, `batch`, `stream`, `ainvoke`, ...). This makes it possible for chains of LCEL objects to also automatically support useful operations like batching and streaming of intermediate steps, since every chain of LCEL objects is itself an LCEL object.
2. **Composition primitives**: LCEL provides a number of primitives that make it easy to compose chains, parallelize components, add fallbacks, dynamically configure chain internals, and more.

LangChain maintains a number of legacy abstractions. Many of these can be easily implemented via short combinations of LCEL primitives. Doing so confers some advantages:

- The resulting chains typically implement the full `Runnable` interface, including streaming and asynchronous support where appropriate;
- The chains may be more easily extended or modified;
- The parameters of the chain are typically surfaced for easier customization (e.g., prompts).

In this guide we review LCEL implementations of common legacy abstractions. Where appropriate, we link out to separate guides with more detail.

In [ ]:
%pip install --upgrade --quiet  langchain-core langchain langchain-openai

## LLMChain
[LLMChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html) composes a prompt template, LLM, and output parser.

<ColumnContainer>

<Column>

#### Legacy


In [17]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)


</Column>

<Column>

#### LCEL



In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = prompt | ChatOpenAI() | StrOutputParser()


</Column>
</ColumnContainer>

Note that `LLMChain` by default returns a `dict` containing both the input and the output. If this behavior is desired, we can replicate it using another LCEL primitive, [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html):

In [21]:
from langchain_core.runnables import RunnablePassthrough

outer_chain = RunnablePassthrough().assign(text=chain)

See [this tutorial](/docs/tutorials/llm_chain) for more detail on building with prompt templates, LLMs, and output parsers.

## ConversationChain

[ConversationChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.conversation.base.ConversationChain.html) incorporates a memory of previous messages to sustain a stateful conversation.


<ColumnContainer>
<Column>

#### Legacy



In [30]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

memory = ConversationBufferMemory()
chain = ConversationChain(llm=ChatOpenAI(), memory=memory)

</Column>

<Column>

#### LCEL



In [39]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

history = InMemoryChatMessageHistory()
chain = RunnableWithMessageHistory(ChatOpenAI(), lambda x: history)

See [this tutorial](/docs/tutorials/chatbot) for more detail on building with [RunnableWithMessageHistory](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html).

Note that `ConversationChain` has no notion of threads, or separate sessions. `RunnableWithMessageHistory` implements this concept via configuration parameters. It should be instantiated with a callable that returns a [chat message history](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html). By default, it expects this function to take a single argument `session_id`:

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


chain = RunnableWithMessageHistory(ChatOpenAI(), get_session_history)

chain.invoke("Hello!", config={"configurable": {"session_id": "abc123"}})

</Column>
</ColumnContainer>


## ConversationRetrievalChain
